# ezlocalai

Open this notebook in [Google Colab.](https://colab.research.google.com/)

Set the resources to T4 GPU, free tier.

It takes several minutes to install dependencies.


In [ ]:
!apt-get update --fix-missing && apt-get upgrade -y
!apt-get install -y --fix-missing --no-install-recommends git build-essential cmake gcc g++ portaudio19-dev ffmpeg libportaudio2 libasound-dev python3 python3-pip wget ocl-icd-opencl-dev opencl-headers clinfo libclblast-dev libopenblas-dev ninja-build python3.10-dev
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!ln -s /usr/bin/python3 /usr/bin/python
!apt-get clean && rm -rf /var/lib/apt/lists/* /var/cache/apt/* /tmp/* /var/tmp/*
!python3 -m pip install --upgrade pip cmake scikit-build setuptools wheel pyngrok --no-cache-dir
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir
!rm -rf sample_data .config
!git clone https://github.com/DevXT-LLC/ezlocalai .
!pip install -r cuda-requirements.txt

## Start the server

Set the `NGROK_TOKEN` to use NGROK to expose your ezlocalai server to the public with as simple as an API key. [Get your free NGROK_TOKEN here.](https://dashboard.ngrok.com/get-started/your-authtoken)

You can see the ngrok URL in the cell output after running the cell.

Set the `LLM_TO_USE` with the name of the model from the models list. We will use `phi-2-dpo` for this example since it is a smaller model that can still be useful.


In [ ]:
LLM_TO_USE = "phi-2-dpo"

# Add your NGROK_TOKEN to your colab secrets if using Google Colab (Key logo on the left)
try:
    from google.colab import userdata
    NGROK_TOKEN = userdata.get('NGROK_TOKEN')
    if not NGROK_TOKEN:
        raise
except:
    # If you're not using Google Colab, enter your NGROK_TOKEN below.
    NGROK_TOKEN = "Enter your ngrok token here"
if NGROK_TOKEN != "Enter your ngrok token here":
    with open('.env', 'r') as file:
        filedata = file.read()
    filedata = filedata.replace('NGROK_TOKEN=\n', f'NGROK_TOKEN={NGROK_TOKEN}\n')
    filedata = filedata.replace('DEFAULT_MODEL=phi-2-dpo', f'DEFAULT_MODEL={LLM_TO_USE}')
    with open('.env', 'w') as file:
        file.write(filedata)
!uvicorn app:app --host 0.0.0.0 --port 8091 --workers 1 --proxy-headers